In [ ]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('restaurant_customer_satisfaction.csv')

percentage = 0.3
data[:int(len(data)*percentage):]

In [ ]:
# remove unused columns
data = data.drop(columns=['CustomerID','Age','Gender','Income','VisitFrequency','PreferredCuisine','TimeOfVisit','GroupSize','DiningOccasion',
'MealType','OnlineReservation','DeliveryOrder','LoyaltyProgramMember','FoodRating','AmbianceRating','HighSatisfaction'])

# save data to csv
data.to_csv('cleaned_data.csv', index=False)

In [ ]:
# data = pd.read_csv('cleaned_data.csv')

# # sort data by WaitTime
# data = data.sort_values(by='WaitTime', ascending=True)
# data.head()

# # sort data by ServiceRating
# data = data.sort_values(by='ServiceRating', ascending=True)
# data.head()

# # sort data by AverageSpend
# data = data.sort_values(by='AverageSpend', ascending=True)
# data.head()


In [ ]:
# use data
data = pd.read_csv('cleaned_data.csv')

# Define the universal variables
WaitTime = ctrl.Antecedent(np.arange(0, 60 + 1, 1), 'WaitTime')
ServiceRating = ctrl.Antecedent(np.arange(0, 6, 1), 'ServiceRating')
AverageSpend = ctrl.Antecedent(np.arange(0, 200 + 1, 1), 'AverageSpend')
satisfaction = ctrl.Consequent(np.arange(0, 6, 1), 'satisfaction')

# WaitTime Membership Functions
WaitTime['low'] = fuzz.trapmf(WaitTime.universe, [0, 0, 20, 30])
WaitTime['medium'] = fuzz.trimf(WaitTime.universe, [20, 30, 40])
WaitTime['high'] = fuzz.trapmf(WaitTime.universe, [30, 40, 60, 60])

# ServiceRating Membership Functions
ServiceRating['low'] = fuzz.trapmf(ServiceRating.universe, [0, 0, 2, 3])
ServiceRating['medium'] = fuzz.trimf(ServiceRating.universe, [2, 3, 4])
ServiceRating['high'] = fuzz.trapmf(ServiceRating.universe, [3, 4, 5, 5])

# AverageSpend Membership Functions
AverageSpend['low'] = fuzz.trapmf(AverageSpend.universe, [0, 0, 50, 100])
AverageSpend['average'] = fuzz.trimf(AverageSpend.universe, [50, 100, 150])
AverageSpend['high'] = fuzz.trapmf(AverageSpend.universe, [100, 150, 200, 200])

# satisfaction Membership Functions
satisfaction['low'] = fuzz.trapmf(satisfaction.universe, [0, 0, 2, 3])
satisfaction['medium'] = fuzz.trimf(satisfaction.universe, [2, 3, 4])
satisfaction['high'] = fuzz.trapmf(satisfaction.universe, [3, 4, 5, 5])

WaitTime.view()
ServiceRating.view()
AverageSpend.view()
satisfaction.view()
plt.show()

In [ ]:
# Rule Base
rule1 = ctrl.Rule(WaitTime['low'] & ServiceRating['low'] & AverageSpend['low'], satisfaction['low'])
rule2 = ctrl.Rule(WaitTime['low'] & ServiceRating['low'] & AverageSpend['average'], satisfaction['medium'])
rule3 = ctrl.Rule(WaitTime['low'] & ServiceRating['low'] & AverageSpend['high'], satisfaction['high'])
rule4 = ctrl.Rule(WaitTime['low'] & ServiceRating['medium'] & AverageSpend['low'], satisfaction['low'])
rule5 = ctrl.Rule(WaitTime['low'] & ServiceRating['medium'] & AverageSpend['average'], satisfaction['medium'])
rule6 = ctrl.Rule(WaitTime['low'] & ServiceRating['medium'] & AverageSpend['high'], satisfaction['high'])
rule7 = ctrl.Rule(WaitTime['low'] & ServiceRating['high'] & AverageSpend['low'], satisfaction['low'])
rule8 = ctrl.Rule(WaitTime['low'] & ServiceRating['high'] & AverageSpend['average'], satisfaction['medium'])
rule9 = ctrl.Rule(WaitTime['low'] & ServiceRating['high'] & AverageSpend['high'], satisfaction['high'])
rule10 = ctrl.Rule(WaitTime['medium'] & ServiceRating['low'] & AverageSpend['low'], satisfaction['low'])
rule11 = ctrl.Rule(WaitTime['medium'] & ServiceRating['low'] & AverageSpend['average'], satisfaction['medium'])
rule12 = ctrl.Rule(WaitTime['medium'] & ServiceRating['low'] & AverageSpend['high'], satisfaction['high'])
rule13 = ctrl.Rule(WaitTime['medium'] & ServiceRating['medium'] & AverageSpend['low'], satisfaction['low'])
rule14 = ctrl.Rule(WaitTime['medium'] & ServiceRating['medium'] & AverageSpend['average'], satisfaction['medium'])
rule15 = ctrl.Rule(WaitTime['medium'] & ServiceRating['medium'] & AverageSpend['high'], satisfaction['high'])
rule16 = ctrl.Rule(WaitTime['medium'] & ServiceRating['high'] & AverageSpend['low'], satisfaction['low'])
rule17 = ctrl.Rule(WaitTime['medium'] & ServiceRating['high'] & AverageSpend['average'], satisfaction['medium'])
rule18 = ctrl.Rule(WaitTime['medium'] & ServiceRating['high'] & AverageSpend['high'], satisfaction['high'])
rule19 = ctrl.Rule(WaitTime['high'] & ServiceRating['low'] & AverageSpend['low'], satisfaction['low'])
rule20 = ctrl.Rule(WaitTime['high'] & ServiceRating['low'] & AverageSpend['average'], satisfaction['medium'])
rule21 = ctrl.Rule(WaitTime['high'] & ServiceRating['low'] & AverageSpend['high'], satisfaction['high'])
rule22 = ctrl.Rule(WaitTime['high'] & ServiceRating['medium'] & AverageSpend['low'], satisfaction['low'])
rule23 = ctrl.Rule(WaitTime['high'] & ServiceRating['medium'] & AverageSpend['average'], satisfaction['medium'])
rule24 = ctrl.Rule(WaitTime['high'] & ServiceRating['medium'] & AverageSpend['high'], satisfaction['high'])
rule25 = ctrl.Rule(WaitTime['high'] & ServiceRating['high'] & AverageSpend['low'], satisfaction['low'])
rule26 = ctrl.Rule(WaitTime['high'] & ServiceRating['high'] & AverageSpend['average'], satisfaction['medium'])
rule27 = ctrl.Rule(WaitTime['high'] & ServiceRating['high'] & AverageSpend['high'], satisfaction['high'])

# Build the control system
satisfaction_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9,
rule10, rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20, rule21, rule22, rule23, rule24, rule25, rule26, rule27])

# Function to compute satisfaction
def compute_satisfaction(row):
    try:
        # Create a new simulation instance for each row
        satisfaction_simulation = ctrl.ControlSystemSimulation(satisfaction_ctrl)
        
        # Get input values
        wait_time = row['WaitTime']
        service = row['ServiceRating']
        spend = row['AverageSpend']
        
        # Set inputs
        satisfaction_simulation.input['WaitTime'] = wait_time
        satisfaction_simulation.input['ServiceRating'] = service
        satisfaction_simulation.input['AverageSpend'] = spend
        
        # Compute the fuzzy result
        satisfaction_simulation.compute()
        
        # Return the satisfaction value
        return satisfaction_simulation.output['satisfaction']
    
    except Exception as ex:
        print(f"Error computing for row {row.name}: {ex}")
        print(f"Inputs: WaitTime={wait_time}, ServiceRating={service}, AverageSpend={spend}")
        return np.nan

# Apply the fuzzy logic model to the dataset
data['satisfaction'] = data.apply(compute_satisfaction, axis=1)

# Check for NaN values
nan_count = data['satisfaction'].isna().sum()
print(f"Number of rows with NaN satisfaction: {nan_count}")

In [ ]:
# Define a function to classify the movies
def classify_satisfaction(score):
    if score >= 4.0:
        return 'High'
    elif score >= 2.5:
        return 'Medium'
    else:
        return 'Low'

# Display the satisfaction with the highest quality scores
finaldata = data.sort_values(by='satisfaction', ascending=False)

# Check the size of the final dataset
print(f"Final dataset size: {len(finaldata)} rows")

# Display the first 10 rows
print("\nTop 10 satisfaction scores:")
print(finaldata[['WaitTime', 'ServiceRating', 'AverageSpend', 'satisfaction']].head(10))

# Save the full satisfaction data to a csv file
finaldata.to_csv('satisfaction.csv', index=False)

# Verify the saved file
saved_data = pd.read_csv('satisfaction.csv')
print(f"\nSaved CSV file contains {len(saved_data)} rows")

In [ ]:
# Plot the distribution of movie quality scores
plt.figure(figsize=(10,6))
plt.hist(data['movie_quality_score'].dropna(), bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Movie Quality Scores')
plt.xlabel('Movie Quality Score')
plt.ylabel('Number of Movies')
plt.grid(True)
plt.show()

In [ ]:
# Count the number of movies in each category
category_counts = data['movie_quality_category'].value_counts()
print("Movie Quality Category Counts:")
print(category_counts)

# Plot the counts
category_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Number of Movies in Each Quality Category')
plt.xlabel('Movie Quality Category')
plt.ylabel('Number of Movies')
plt.grid(True)
plt.show()